# 🚀 Digital Sahayak AI - Training Notebook
## Fine-tune your own custom AI model

This notebook will help you train your own AI model for Digital Sahayak.

**Requirements:**
- Google Colab with GPU (T4 free tier works, A100 recommended)
- Your training data in JSON format
- ~15GB disk space

## Step 1: Install Dependencies

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets accelerate peft bitsandbytes trl sentencepiece
!pip install -q huggingface_hub

print("✅ All dependencies installed!")

## Step 2: Login to Hugging Face (for downloading base model)

In [ ]:
from huggingface_hub import login

# Get your token from: https://huggingface.co/settings/tokens
# Make sure you have accepted Mistral model terms
HF_TOKEN = "hf_your_token_here"  # <-- Replace with your token

login(token=HF_TOKEN)
print("✅ Logged in to Hugging Face!")

## Step 3: Configuration

In [ ]:
# ========== CONFIGURATION ==========
# Change these as per your needs

MODEL_CONFIG = {
    # Your custom model name
    "output_model_name": "Pratham-ML",
    
    # Training data path (upload your JSON file)
    "training_data": "train_data.json",
    
    # Training parameters
    "num_epochs": 3,
    "batch_size": 2,
    "gradient_accumulation": 8,
    "learning_rate": 2e-4,
    "max_seq_length": 2048,
    
    # LoRA parameters
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    
    # Output directory
    "output_dir": "./output",
}

print(f"🎯 Training: {MODEL_CONFIG['output_model_name']}")
print(f"📊 Epochs: {MODEL_CONFIG['num_epochs']}")
print(f"📁 Output: {MODEL_CONFIG['output_dir']}")

## Step 4: Prepare Training Data

Your training data should be in this format:
```json
[
  {"instruction": "PM-KISAN yojana kya hai?", "response": "PM-KISAN ek sarkari yojana hai..."},
  {"instruction": "SSC MTS ki eligibility batao", "response": "SSC MTS ke liye 10th pass hona chahiye..."}
]
```

In [ ]:
import json
from datasets import Dataset

# Sample training data (replace with your own)
SAMPLE_DATA = [
    {
        "instruction": "PM-KISAN yojana kya hai?",
        "response": "PM-KISAN (Pradhan Mantri Kisan Samman Nidhi) ek sarkari yojana hai jisme kisano ko har saal ₹6,000 ki madad milti hai. Yeh rashi 3 kisht mein ₹2,000-₹2,000 karke di jaati hai. Is yojana ki shuruat 2018 mein hui thi."
    },
    {
        "instruction": "SSC MTS ki eligibility kya hai?",
        "response": "SSC MTS (Multi Tasking Staff) ke liye eligibility:\n- Education: 10th Pass (Matriculation)\n- Age: 18-25 years\n- Age relaxation: OBC - 3 years, SC/ST - 5 years\n\nWebsite: ssc.nic.in"
    },
    {
        "instruction": "Bihar ka capital kya hai?",
        "response": "Bihar ki rajdhani Patna hai. Patna ek bahut purana sheher hai aur Ganga nadi ke kinare basaa hai."
    },
    {
        "instruction": "Aadhar card kaise banwaye?",
        "response": "Aadhar card banwane ke liye:\n1. Apne najdiki Aadhar Enrollment Center jaye\n2. Required documents le jaaye (ID proof, address proof, DOB proof)\n3. Biometric (fingerprint, iris scan) dein\n4. Form bhare\n5. 90 din mein Aadhar post se aa jayega\n\nOnline appointment: uidai.gov.in"
    },
    {
        "instruction": "Good morning",
        "response": "🌅 Suprabhat! Aapka din shubh ho! Main Digital Sahayak AI hoon. Aaj kisme madad karoon? Sarkari yojana, naukri, ya documents - kuch bhi poochiye!"
    },
    {
        "instruction": "Ayushman Bharat yojana kab shuru hui?",
        "response": "Ayushman Bharat yojana 23 September 2018 ko shuru hui thi. Ise PM Narendra Modi ne Jharkhand ke Ranchi se launch kiya tha. Is yojana mein garib parivaron ko ₹5 lakh tak ka free health insurance milta hai."
    }
]

# Save sample data
with open(MODEL_CONFIG['training_data'], 'w', encoding='utf-8') as f:
    json.dump(SAMPLE_DATA, f, ensure_ascii=False, indent=2)

print(f"✅ Sample training data saved: {len(SAMPLE_DATA)} examples")
print("\n⚠️ Upload your own 'train_data.json' with more examples for better results!")

In [ ]:
# Load and prepare dataset
import json

with open(MODEL_CONFIG['training_data'], 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Format for training
def format_prompt(example):
    return f"""### Instruction:
{example['instruction']}

### Response:
{example['response']}"""

formatted_data = [{"text": format_prompt(d)} for d in raw_data]
dataset = Dataset.from_list(formatted_data)

print(f"✅ Dataset loaded: {len(dataset)} examples")
print(f"\n📝 Sample:\n{dataset[0]['text'][:300]}...")

## Step 5: Load Base Model with 4-bit Quantization

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Base model (Apache 2.0 licensed)
BASE_MODEL = "mistralai/Mistral-7B-v0.1"

# 4-bit quantization config (saves memory)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("📥 Loading base model (this takes 2-5 minutes)...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

print("✅ Base model loaded successfully!")
print(f"📊 Model size: ~{model.num_parameters()/1e9:.1f}B parameters")

## Step 6: Apply LoRA Adapters

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=MODEL_CONFIG['lora_r'],
    lora_alpha=MODEL_CONFIG['lora_alpha'],
    lora_dropout=MODEL_CONFIG['lora_dropout'],
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA adapters applied!")
print(f"📊 Trainable parameters: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
print(f"📊 Total parameters: {total_params:,}")

## Step 7: Training

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir=MODEL_CONFIG['output_dir'],
    num_train_epochs=MODEL_CONFIG['num_epochs'],
    per_device_train_batch_size=MODEL_CONFIG['batch_size'],
    gradient_accumulation_steps=MODEL_CONFIG['gradient_accumulation'],
    learning_rate=MODEL_CONFIG['learning_rate'],
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    max_grad_norm=0.3,
    report_to="none",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=MODEL_CONFIG['max_seq_length'],
    packing=False,
)

print("🚀 Starting training...")
print(f"📊 Total steps: {len(trainer.get_train_dataloader()) * MODEL_CONFIG['num_epochs']}")

In [ ]:
# Train!
trainer.train()

print("\n✅ Training completed!")

## Step 8: Save LoRA Adapters

In [ ]:
# Save the LoRA adapters
adapter_path = f"{MODEL_CONFIG['output_dir']}/lora_adapters"
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)

print(f"✅ LoRA adapters saved to: {adapter_path}")

## Step 9: Merge Adapters with Base Model

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch

print("🔄 Merging adapters with base model...")

# Load base model in full precision for merging
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Load and merge LoRA adapters
merged_model = PeftModel.from_pretrained(base_model, adapter_path)
merged_model = merged_model.merge_and_unload()

print("✅ Adapters merged successfully!")

In [ ]:
# Save the final merged model
final_model_path = f"{MODEL_CONFIG['output_dir']}/{MODEL_CONFIG['output_model_name']}"

merged_model.save_pretrained(final_model_path, safe_serialization=True)
tokenizer.save_pretrained(final_model_path)

print(f"✅ Final model saved to: {final_model_path}")

## Step 10: Create LICENSE and Model Card

In [ ]:
# Create LICENSE file (Apache 2.0 requirement)
LICENSE_TEXT = """Apache License
Version 2.0, January 2004
http://www.apache.org/licenses/

Copyright 2023 Mistral AI

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

with open(f"{final_model_path}/LICENSE", 'w') as f:
    f.write(LICENSE_TEXT)

print("✅ LICENSE file created")

In [ ]:
# Create Model Card (README.md)
MODEL_CARD = f"""---
license: apache-2.0
language:
  - en
  - hi
tags:
  - text-generation
  - conversational
  - hindi
  - government-services
---

# {MODEL_CONFIG['output_model_name']}

A conversational AI model for Indian government services assistance.

## Model Details

- **Model Name:** {MODEL_CONFIG['output_model_name']}
- **Languages:** Hindi, English
- **Use Case:** Government schemes, jobs, documents assistance
- **License:** Apache 2.0

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{MODEL_CONFIG['output_model_name']}")
tokenizer = AutoTokenizer.from_pretrained("{MODEL_CONFIG['output_model_name']}")

prompt = "PM-KISAN yojana kya hai?"
inputs = tokenizer(f"### Instruction:\\n{{prompt}}\\n\\n### Response:\\n", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

## Training

This model was trained using LoRA (Low-Rank Adaptation) technique.

## Disclaimer

This model is for informational purposes. Always verify information from official government sources.
"""

with open(f"{final_model_path}/README.md", 'w') as f:
    f.write(MODEL_CARD)

print("✅ Model card (README.md) created")

## Step 11: Test the Model

In [ ]:
# Test the trained model
def generate_response(prompt, max_tokens=256):
    formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(merged_model.device)
    
    outputs = merged_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:\n")[-1].strip()

# Test queries
test_queries = [
    "Good morning",
    "PM-KISAN yojana kya hai?",
    "SSC MTS ki eligibility batao",
    "Bihar ka capital kya hai?"
]

print("🧪 Testing the model:\n")
for query in test_queries:
    print(f"❓ User: {query}")
    response = generate_response(query)
    print(f"🤖 AI: {response}")
    print("-" * 50)

## Step 12: Download the Model

In [ ]:
# Zip the model for download
import shutil

zip_path = f"{MODEL_CONFIG['output_model_name']}.zip"
shutil.make_archive(MODEL_CONFIG['output_model_name'], 'zip', final_model_path)

print(f"✅ Model zipped: {zip_path}")
print(f"📥 Download from Colab file browser or use:")
print(f"   from google.colab import files")
print(f"   files.download('{zip_path}')")

In [ ]:
# Optional: Upload to Hugging Face Hub
# Uncomment if you want to upload

# from huggingface_hub import HfApi
# 
# api = HfApi()
# api.create_repo(repo_id=f"your-username/{MODEL_CONFIG['output_model_name']}", private=True)
# api.upload_folder(
#     folder_path=final_model_path,
#     repo_id=f"your-username/{MODEL_CONFIG['output_model_name']}",
# )
# print("✅ Model uploaded to Hugging Face Hub!")

---
## 🎉 Done!

Your custom **Pratham-ML** model is ready!

### What you got:
1. ✅ Fine-tuned model weights (`.safetensors`)
2. ✅ Tokenizer files
3. ✅ LICENSE file (Apache 2.0 compliance)
4. ✅ Model card (README.md)

### Next steps:
1. Download the model zip file
2. Extract and use in your application
3. Or upload to Hugging Face Hub for easy access

### Tips for better results:
- Add more training data (500+ examples recommended)
- Include diverse question types
- Train for more epochs (3-5)
- Use A100 GPU for faster training